In [1]:
import dask
import dask.dataframe as ddf
import os
from glob import glob
import re
import pandas as pd
import numpy as np
import pickle

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
a = ddf.read_parquet('../../../../intermediate-result-new-select-editors/editor-profile/amr-info-per-editor-per-page/')
a = a.compute()

In [3]:
b = df = pd.read_parquet('../../../../intermediate-result-new-select-editors/editor-profile/contribution-info/')

In [4]:
c = pd.read_parquet('../../../../intermediate-result-new-select-editors/editor-profile/later-info/')

In [5]:
d = pd.read_parquet('../../../../intermediate-result-new-select-editors/editor-profile/revert-info/')

In [6]:
e  = pd.read_parquet('/home/ubuntu/scratch/xinrui/intermediate-result/all-data/sig-contributions-all-data/')

In [7]:
f = pd.read_parquet('/home/ubuntu/scratch/xinrui/intermediate-result-new-select-editors/editor-profile/ref-info-with-title-editor/')

In [8]:
with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as fb:
    selected_articles = pickle.load(fb)

In [9]:
selected_talks = {'Talk:' + i for i in selected_articles}

In [10]:
selected_articles.update(selected_talks)

In [11]:
a = a[a['page.title'].isin(selected_articles)]

In [12]:
b = b[b['page.title'].isin(selected_articles)]
c = c[c['page.title'].isin(selected_articles)]
d = d[d['page.title'].isin(selected_articles)]

In [13]:
e = e[e['page.title'].isin(selected_articles)]
f = f[f['page.title'].isin(selected_articles)]

In [14]:
a = a.groupby(['contributor.username','page.title']).sum()

In [15]:
b = b.groupby(['contributor.username','page.title']).sum()

In [16]:
c = c.groupby(['contributor.username','page.title']).sum()

In [17]:
d = d.groupby(['contributor.username','page.title']).sum()

In [18]:
e = e.groupby(['contributor.username','page.title']).count()

In [19]:
f = f.groupby(['contributor.username','page.title']).sum()

In [20]:
article_fin_info = a.join(b).join(c).join(d).join(e).join(f).fillna(0)

In [21]:
article_fin_info = article_fin_info.rename(columns={'count':'total.contributions','added_len':'sig.contributions'})

In [22]:
def pivit_df2(metric,df):
    user_edits= df.pivot_table(index='contributor.username', columns='page.title',
                         values=metric, aggfunc='first').fillna(0).astype('int')
    user_edits.index = metric + '.' + user_edits.index
    return user_edits

In [23]:
metrics = ['text.added', 'text.modified', 'text.removed',
       'wikilinks.added', 'wikilinks.modified', 'wikilinks.removed', 'url.added',
       'url.modified', 'url.removed','total.contributions','reverts','sig.contributions','rules',
          'arbcom','ban','refs.added','refs.modified','refs.removed']

In [24]:
res = {}
for item in metrics:
    res[item] = pivit_df2(item,article_fin_info)

In [25]:
fin_t = pd.concat(res).reset_index().set_index('contributor.username').drop(columns={'level_0'})

In [26]:
fin_t = fin_t.loc[~(fin_t==0).all(axis=1)]
fin_t = fin_t.loc[:, ~(fin_t == 0).all(axis=0)]

In [27]:
article_c_score = pd.read_parquet('/home/ubuntu/scratch/xinrui/intermediate-result/article_c_score')

In [28]:
article_list = fin_t.columns.to_list()

In [29]:
c_score_article = pd.DataFrame({'page.title':article_list}).set_index('page.title').join(article_c_score)

In [30]:
c_score_article = c_score_article.rename(columns={'estimated_crc':'article.c_score'})

In [31]:
c_score_article_T = c_score_article.T

In [32]:
fin_t = pd.concat([fin_t,c_score_article_T])

In [33]:
fin_t.to_csv("result/article-profile.tsv", sep="\t",encoding='utf-16')

In [34]:
fin_t_df = ddf.from_pandas(fin_t,chunksize=1000)

In [35]:
fin_t_df.to_parquet('result/article-profile')